# Автогенерация текстовых описаний к видео (кейс Rutube)

В данном кейсе вам предлагается решить задачу автогенерации краткого тектового описания к видео, на основе видеофайла и автоматической транскрибации. 

Структура датасета следующая:

train.csv
- **video_name** - название видео (в директории **train_video**)
- **stt_name** - название файла с транскрибацией (в директории **train_stt**)
- **category_name** - категория видео
- **title** - название видео
- **description** - описание видео

В ноутбуке вы можете пронаблюдать baseline модель, без обучения (unsupervised) в качестве простого примера, основанную только на файле транскрибации. Также в конце считается метрика meteor по baseline модели и модели, которая из транскрипта речи (STT) выдает первые несколько предложений для сравнения. 

Тестовый датасет будет прислан вам позднее, поэтому здесь он фигурировать не будет.

Немного про модель: LexRankSummarizer, не вдаваясь в детали, можно сказать, что модель основана на статистиках, ее цель - найти самые "важные" предложения из полного текста (STT). 

Предложения представляются в виде мешка слов и получают эмбеддинги c tfidf, далее считаются косинусные близости предложений друг с другом. Следующая часть модели взята из немалоизвестной PageRank - строится граф, где на рёбрах стоит косинусная близость. Финальная часть  - по графу строится матрица, в ней находится максимальное сингулярное значение и таким образом находятся самые "значимые" предложения из большого текста.

Подробнее можно почитать например тут https://www.codingninjas.com/studio/library/lexrank

На метрики и сравнение моделей на других бенчмарках тут https://www.dialog-21.ru/media/5764/golovizninavspluskotelnikovev038.pdf

Для предобработки данных мы только удаляем стоп-слова (слишком часто встречаемые, например предлоги, союзы и тп), которые могут портить модель.

## Предобученная T5

In [ ]:
working_dir = '/kaggle/input/rut-data/rutube_hackathon_summary_generation_novosibirsk/'

In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split


dataset_train = pd.read_csv(os.path.join(working_dir, "train", "train.csv"))
dataset_test = pd.read_csv(os.path.join(working_dir, "test", "test.csv"))

In [ ]:
tqdm.pandas()

def del_timestamps(text):
    text = text.split("]  ")[1:]
    return " ".join(text)

def cut(lines: str, max_words=512):
    if len(lines.split()) > max_words:
        lines = " ".join(lines.split()[:max_words])
    return lines
    


def gen_description(stt_name, n_sent, category_name, mode='train'):

    with open(os.path.join(working_dir, mode, f'{mode}_stt', stt_name), 'r') as f:
        lines = f.readlines()
        lines = [del_timestamps(line.strip()) for line in lines]
        lines = " ".join(lines)
        res = cut(lines)
        if len(res)>0:
            return res
        else:
            return category_name
        
        
dataset_train['stt_sum'] = np.nan
dataset_train['stt_sum'] = dataset_train.progress_apply(lambda l: gen_description(l.stt_name, 4, l.category_name, 'train'), axis=1)

dataset_test['stt_sum'] = np.nan
dataset_test['stt_sum'] = dataset_test.progress_apply(lambda l: gen_description(l.stt_name, 4, l.category_name, 'test'), axis=1)

dataset_train, dataset_valid = train_test_split(dataset_train, test_size=0.1, shuffle=True)
print(f'Train: {len(dataset_train)}\nValid: {len(dataset_valid)}\nTest: {len(dataset_test)}')

In [ ]:
!pip install -U transformers


In [ ]:
import torch
import pprint
import numpy as np
 
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
)


In [ ]:
MODEL = 'cointegrated/rut5-base-multitask'
BATCH_SIZE = 64
EPOCHS = 2
MAX_LENGTH = 512

In [ ]:
tokenizer = T5Tokenizer.from_pretrained(MODEL)
 
# Function to convert text data into model inputs and targets
def preprocess_function(examples):
    inputs = [f"summarize: {article}" for article in examples['stt_sum']]
    tokenized_inputs = tokenizer(
        inputs,
        max_length=MAX_LENGTH,
        truncation=True,
        padding='max_length',  
        return_tensors="pt"
    )
 
    # Set up the tokenizer for targets
    targets = [summary for summary in examples['description']]
    with tokenizer.as_target_tokenizer():
        tokenized_targets = tokenizer(
            targets,
            max_length=MAX_LENGTH,
            truncation=True,
            padding='max_length',
            return_tensors="pt"
        ).input_ids
 
    return tokenized_inputs.input_ids, tokenized_targets, tokenized_inputs.attention_mask
 
tokenized_train_inputs, tokenized_train_targets, attention_mask_train = preprocess_function(dataset_train)
tokenized_val_inputs, tokenized_val_targets, attention_mask_val = preprocess_function(dataset_valid)


In [ ]:
from torch.utils.data import DataLoader, Dataset

class CustomDataset(Dataset):
    def __init__(self, inputs, targets, attn):
        self.inputs, self.targets, self.attn = inputs, targets, attn


    def __len__(self):
        return len(self.inputs)

    
    def __getitem__(self, idx):
        return (self.inputs[idx], self.targets[idx], self.attn[idx])
    
train_custom_dataset = CustomDataset(tokenized_train_inputs, tokenized_train_targets, attention_mask_train)
val_custom_dataset = CustomDataset(tokenized_val_inputs, tokenized_val_targets, attention_mask_val)


train_dataloader = DataLoader(train_custom_dataset, batch_size=8)
val_dataloader = DataLoader(val_custom_dataset, batch_size=8)

In [ ]:
import torch
@torch.no_grad()
def evaluate(model, criterion, val_loader):
    model.eval()
    losses = 0
    for src, tgt, attn in tqdm(val_loader, total=len(list(val_dataloader))):
        src = src.to(device)
        tgt = tgt.to(device)
        attn = attn.to(device)
        
        tgt[tgt == tokenizer.pad_token_id] = -100
        loss = model(input_ids=src, labels=tgt, attention_mask=attn).loss        
        
        losses += loss.item()
        torch.cuda.empty_cache()
    return losses / len(list(val_loader))

In [ ]:
import gc
def train_epoch(model, optimizer, train_loader):
    model.train()
    losses = 0
    for src, tgt, attn in tqdm(train_dataloader, total=len(list(train_loader))):            
    
        tgt[tgt == tokenizer.pad_token_id] = -100
        
        src = src.to(device)
        tgt = tgt.to(device)
        attn = attn.to(device)
        loss = model(input_ids=src, labels=tgt, attention_mask=attn).loss
        print(loss)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        
        losses += loss.item()
        torch.cuda.empty_cache()
        del src
        del tgt
        del attn
        gc.collect()
    return losses / len(list(train_loader))


def train(model, optimizer, scheduler, train_loader, val_loader, n_epochs):
    train_loss = 0.0
    val_loss = 0.0
    for epoch in range(n_epochs):
        train_loss = train_epoch(model, optimizer, train_loader)
        val_loss = evaluate(model, val_loader)
        print(f"Epoch {epoch}\ntrain loss: {train_loss}\nval loss: {val_loss}\n")

        if scheduler is not None:
            scheduler.step(val_loss)
        

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(MODEL)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
model.to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
train(model, optimizer, None, train_dataloader, val_dataloader, 1)

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained(MODEL)
model = T5ForConditionalGeneration.from_pretrained(MODEL)

input_ids = tokenizer("Привет как дела", return_tensors="pt").input_ids
labels = tokenizer("Все хорошо", return_tensors="pt").input_ids

# the forward function automatically creates the correct decoder_input_ids
loss = model(input_ids=tokenized_train_inputs[:64, :], labels=tokenized_train_targets[:64, :]).loss
loss.item()


## Бейзлайны

In [ ]:
working_dir = '/kaggle/input/rut-data/rutube_hackathon_summary_generation_novosibirsk/'

In [ ]:
# lex rank - unsupervised upproach
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.nlp.stemmers import Stemmer
import nltk
from nltk.corpus import stopwords
import numpy as np


# nltk.download('stopwords')

# Допольнительные стоп-слова можно скачать здесь
# https://github.com/stopwords-iso/stopwords-ru/blob/master/raw/stop-words-russian.txt
# но в этот список мы также добавили пару примеров вручную, поэтому прикладываем готовый файл. 
# Вы также можете модифицировать на свое усмотрение, или вовсе от него отказаться

with open(working_dir + "stop-words-russian.txt", 'r') as f:
    extra_stop_words = f.readlines()
    extra_stop_words = [line.strip() for line in extra_stop_words]


def sumy_method(text, n_sent: int = 4):
    
    parser = PlaintextParser.from_string(text, Tokenizer("russian"))
    
    stemmer = Stemmer("russian")
    summarizer = LexRankSummarizer(stemmer)
    stopwords_ru = stopwords.words('russian')
    stopwords_ru.extend(extra_stop_words)
    summarizer.stop_words = stopwords_ru
    
    #Summarize the document with n_sent sentences
    summary = summarizer(parser.document, n_sent)
    dp = []
    if len(summary)> 0:
        for i in summary:
            lp = str(i)
            dp.append(lp)
    
        final_sentence = ' '.join(dp)
    else:
        final_sentence = ''
    if len(final_sentence.split(" "))>512:
        final_sentence = " ".join(final_sentence.split(" ")[:512])
    return final_sentence

In [ ]:
import pandas as pd
import os
PATH_TO_DATA = working_dir + 'train/'
dataset = pd.read_csv(os.path.join(PATH_TO_DATA, "train.csv"))

In [ ]:
dataset.head(5)

In [ ]:
with open(os.path.join(PATH_TO_DATA, 'train_stt', '478.txt'), 'r') as f:
        lines = f.readlines()
        lines = [line.strip() for line in lines]
lines

### Чтобы понять сколько предложений нам нужно выдавать в качестве описания, посчитаем статистики

In [ ]:
import nltk
# from nltk.translate import meteor
from nltk.translate import meteor_score
from nltk import word_tokenize, sent_tokenize


In [ ]:
dataset['len'] = dataset.description.apply(lambda l : len(sent_tokenize(l)))
print("Среднее число предложений в трейн датасете", np.mean(dataset['len'].to_list()))
print("Медиана", np.median(dataset['len'].to_list()))

### Теперь поймём примерный размер в токенах

In [ ]:
dataset['len_tokens'] = dataset.description.apply(lambda l : len(l.split(" ")))

print("Среднее число слов в трейн датасете", np.mean(dataset['len_tokens'].to_list()))
print("Медиана", np.median(dataset['len_tokens'].to_list()))
print("Максимум", np.max(dataset['len_tokens'].to_list()))

In [ ]:
# поэтому в sumy_method мы добавили ограничение на число слов в сгенерированном тексте 
# (512 слов в нашем случае, решили так ограничить макс 348 слов из трейна)

### Генерируем текстовые описания для всех видео из трейна по текстовому описанию (из Speech To Text)
Если в видео не было речи, то в качестве описания ставим категорию видео

In [ ]:
# Очистим STT от временных кодов
from tqdm import tqdm
tqdm.pandas()
def del_timestamps(text):
    text = text.split("]  ")[1:]
    return " ".join(text)

In [ ]:
def gen_description(stt_name, n_sent, category_name):

    with open(os.path.join(PATH_TO_DATA, 'train_stt', stt_name), 'r') as f:
        lines = f.readlines()
        lines = [del_timestamps(line.strip()) for line in lines]
        lines = " ".join(lines)
        res = sumy_method(lines, n_sent)
        if len(res)>0:
            return res
        else:
            return category_name


In [ ]:
%%time
dataset['stt_sum'] = np.nan
dataset['stt_sum'] = dataset.progress_apply(lambda l: gen_description(l.stt_name, 4, l.category_name), axis=1)

In [ ]:
# видео, по которым нет речи и соответсвенно модель не смогла ничего выдать
dataset[dataset.stt_sum.isin(dataset.category_name.unique())]

### Посчитаем метрику meteor

In [ ]:
def func(stt_name, text, text_sum):
    if isinstance(text_sum, str):
        return round(meteor_score.meteor_score([word_tokenize(text)],word_tokenize(text_sum)), 4)
    else:
        return 0
dataset['meteor'] = dataset.apply(lambda l: func(l['stt_name'], l.description, l.stt_sum), axis=1)

In [ ]:
print("Значение метрики meteor для unsupervised модели", dataset.meteor.mean())

### Сравним с моделью, которая выдает первые 4 предложения из STT

In [ ]:
%%time
def func(stt_name, text, category_name):
    with open(os.path.join(PATH_TO_DATA, 'train_stt', stt_name), 'r') as f:
        lines = f.readlines()
        lines = [del_timestamps(line.strip()) for line in lines]
        res = lines[:4]
    res = " ".join(lines)
    if isinstance(res, str):
        return round(meteor_score.meteor_score([word_tokenize(text)],word_tokenize(res)), 4)
    else:
        return round(meteor_score.meteor_score([word_tokenize(text)],word_tokenize(category_name)), 4)
dataset['meteor_first4'] = dataset.apply(lambda l: func(l['stt_name'], l.description, l.category_name), axis=1)

In [ ]:
print("Значение метрики meteor для модели, выдающей первые 4 предложения", dataset.meteor_first4.mean())